# 1. Import Ntuple and DecayHash

In [1]:
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import sys
sys.path.append('/home/belle/zhangboy/inclusive_R_D/')
import utilities as util
import decayHash
from decayHash import DecayHashMap

Welcome to JupyROOT 6.24/06


In [53]:
# read a signal mc as a pandas dataframe

filename=f'../Samples/Signal_MC_ROEx1/B2D_tau_nu/Ntuples/MC_e_50k_0.parquet'
decayhash=f'../Samples/Signal_MC_ROEx1/B2D_tau_nu/Ntuples/hashmap_MC_e_50k.root'
hashmap2 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)
data = pd.read_parquet(filename, engine="pyarrow")
df, samples=util.get_dataframe_samples(data, 'e')

for name in samples:
    print(f'{name}: {len(samples[name])}')

$D\tau\nu$: 11217
$D^\ast\tau\nu$: 1
$D\ell\nu$: 20
$D^\ast\ell\nu$: 10
$D^{\ast\ast}\tau\nu$: 0
$D^{\ast\ast}\ell\nu$: 8
bkg_fakeDTC: 3181
bkg_fakeB: 990
bkg_continuum: 0
bkg_others: 62


In [ ]:
import pandas
pandas.set_option('display.max_rows', None)
print(data.isna().sum())

In [54]:
pd.set_option('display.max_rows', None)
a=samples[r'bkg_others'][['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','D_mcErrors','B0_mcErrors','e_mcErrors','DecayMode']].value_counts()
a

B0_mcPDG  D_mcPDG  e_mcPDG  e_genMotherPDG  D_mcErrors  B0_mcErrors  e_mcErrors  DecayMode
-511.0     411.0    11.0     15.0           0.0         140.0        2180.0      1            18
 511.0    -411.0   -11.0    -15.0           0.0         136.0        2176.0      1            16
                                                        140.0        2180.0      1            16
-511.0     411.0    11.0     15.0           0.0         136.0        2176.0      1            12
dtype: int64

In [52]:
candidate = samples[r'bkg_others'].query('B0_mcErrors==8').iloc[0]
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values

print(candidate[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','DecayMode','B0_mcErrors','e_mcErrors','e_isBremsCorrected']])
print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
org2 = hashmap2.get_original_decay(*hash_values)
print(org2.to_string())
# hashmap1 = DecayHashMap(decayhash, removeRadiativeGammaFlag=False)
# org1 = hashmap1.get_original_decay(*hash_values)
# print(org1.to_string())

B0_mcPDG               511.0
D_mcPDG               -411.0
e_mcPDG                -11.0
e_genMotherPDG         511.0
DecayMode                  1
B0_mcErrors              8.0
e_mcErrors            2048.0
e_isBremsCorrected       1.0
Name: 4095, dtype: object
Monte Carlo Decay with removed Bremsstrahlung gammas: 
300553
    511
        -411
            321
            -211
            -211
        -11
        12
    511
        -411
            321
            -211
            -211
        -15
            -11
            12
            -16
        16



In [51]:
pd.set_option('display.max_rows', None)
b=df.query('130<B0_mcErrors<160')[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','D_mcErrors','B0_mcErrors','e_mcErrors','DecayMode']].value_counts()
b

B0_mcPDG  D_mcPDG  e_mcPDG  e_genMotherPDG  D_mcErrors  B0_mcErrors  e_mcErrors  DecayMode
-511.0     411.0    11.0     15.0           0.0         140.0        2180.0      1            18
 511.0    -411.0   -11.0    -15.0           0.0         136.0        2176.0      1            16
                                                        140.0        2180.0      1            16
-511.0     411.0    11.0     15.0           0.0         136.0        2176.0      1            12
dtype: int64